In [ ]:
class NeuralNetwork:
              
    def sigmoid(x):
        return 1/(1+np.exp(-x))
    
    def ReLU(x):
        return np.maximum(0.0, x)
    
    def __init__(self, design, step_size=0.01, activation_function=sigmoid, dropout=False, bias=False):
        self.design = design
        self.step_size = step_size
        self.activation_function = activation_function # does not work properly (cannot select ReLU)
        self.bias = bias
        self.dropout = dropout
        self.create_weights()
        self.activation = []
    
    def create_weights(self):
        self.weights = [np.zeros(0)]
        for i in np.arange(len(self.design)-1):
            self.weights.append(np.random.uniform(-1,1,[self.design[i+1], self.design[i]]))
    
    def one_training(self, input_data, target_data):
        correction = self.step_size * error * self.activation[-1] * (1.0 - self.activation[-1]) @ self.activation[-2].T
            print('correction: ' + str(correction))
            self.weights[-1] += correction
            
            # Update wih weights
            for i in np.arange(len(self.design)-2,0,-1): # move backwards through NN
                error = self.weights[i+1].T @ error
                correction = self.step_size * ((error * self.activation[i] * (1.0 - self.activation[i])) @ self.activation[i-1].T)
                self.weights[i] += correction
            
    def train(self, input_data, target_data, n_epoches=1, batch_size=100):
        
        # Divide data into mini-batches
        n_batches = int(len(input_data)/batch_size)
        index = np.repeat(np.arange(0,n_batches,1), batch_size)
        np.random.shuffle(index) # randomize assignment to batch    
        overlap = len(input_data) - len(index) # fill up
        index = np.append(index, index[:overlap])
        
        # Update weights for the mean error of every batch
        for b in np.arange(n_batches):
            print('b: ' + str(b))
            
            # Set up data
            input_batch = input_data[index==b,:,:]
            target_batch = target_data[index==b,:]
            error = np.array([0,0], ndmin=2).T # make compatiple to different n_output_nodes!!!
            
            # Compute error for every data point in mini-batch
            for d in np.arange(batch_size):
                print('d: '+str(d))
                # Convert input matrix and target vector to column vectors
                input_vector = np.array(input_batch[d].flatten(), ndmin=2).T
                target_vector = np.array(target_batch[d], ndmin=2).T
                
                # Compute output/activation of each layer
                self.activation = [] # initialize activation list
                self.activation.append(input_vector)
                for i in np.arange(len(self.design)-1):
                    self.activation.append(self.activation_function(self.weights[i+1] @ self.activation[i]))
                
                # Compute error
                error_data_point = target_vector - self.activation[-1]
                error = np.append(error, error_data_point, axis=1)
                print(error_data_point)
            
            # Compute mean error
            error = np.array(np.mean(error[:,1:], axis=1), ndmin=2).T
            print('mean error: ' + str(error))
            
            # Update who weights
            correction = self.step_size * error * self.activation[-1] * (1.0 - self.activation[-1]) @ self.activation[-2].T
            print('correction: ' + str(correction))
            self.weights[-1] += correction
            
            # Update wih weights
            for i in np.arange(len(self.design)-2,0,-1): # move backwards through NN
                error = self.weights[i+1].T @ error
                correction = self.step_size * ((error * self.activation[i] * (1.0 - self.activation[i])) @ self.activation[i-1].T)
                self.weights[i] += correction
    
    def run(self, input_data):
        
        # Convert data into column vector
        input_vector = np.array(input_data.flatten(), ndmin=2).T
        
        # Compute layer outputs/activations
        self.activation = [] # initialize activation list
        self.activation.append(input_vector)
        for i in np.arange(len(self.design)-1):
            self.activation.append(self.activation_function(self.weights[i+1] @ self.activation[i]))
            
        return self.activation[-1]
    
    def evaluate(self, input_data, target, performance_measure=True):
        pass